In [ ]:
## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
print("Finished installing nemo !")

In [ ]:
import nemo.collections.asr as nemo_asr

# Long-form audio transcription

Long-form audio transcription is an interesting application of ASR. Generally, models are trained on short segments of 15-20 seconds of audio clips. If an ASR model is compatibile with streaming inference, it can then be evaluated on audio clips much longer than the training duration.

Generally, streaming inference will incur a small increase in WER due to loss of long term context. Think of it as this - if a streaming model has a context window of a few seconds of audio, even if it streams several minute long audio clips - later transcriptions have lost some of prior context.

-------

In this demo, we consider the naive case of long-form audio transcription, asking the question - in the offline mode (i.e. when the model is given the entire audio sequence at once), what is maximum duration of audio that it can transcribe?

For the purposes of this demo, we will test the limits of Citrinet models [(arxiv)](https://arxiv.org/abs/2104.01721), which are purely convolutional ASR models.

Unlike general attention based models, convolutional models don't have a quadratic cost to their context window, but they also miss out on global context offered by the attention mechanism. Citrinet instead attains relatively long context by replacing attention with [Squeeze-and-Excitation modules](https://arxiv.org/abs/1709.01507) between its blocks.

# Transcribing a podcast

In order to make the task slightly more difficult, we will attempt to transcribe an entire podcast at once. 

Why a podcast? Podcasts are generally long verbal discussions between one or more people on a specific topic, the domain of discussion is unlikely to match the model's training corpus (unless the training corpus is vast), and possibly inclue background audio or sponsorship information in between the discussion.

------

We refer to the following post [Transcription API comparison: Google Speech-to-text, Amazon, Rev.ai](https://cloudcompiled.com/blog/transcription-api-comparison/amp/), where there is a thorough discussion of streaming performance of various cloud providers. Of the three podcasts discussed there, "The staying power of Kubernetes with Kelsey Hightower" is a particularly good match for a use case.

The podcast is somewhat long (nearly 42 minutes), discusses a technical topic (Kubernetes), and provides a detailed transcript of the discussion - including the sponsorship information. This allows a fair comparison between the model's transcription and the actual ground truth - since the model will simply transcribe the sponsorship segments along with the actual discussion.

------
The podcast is available at : https://www.lastweekinaws.com/podcast/screaming-in-the-cloud/the-staying-power-of-kubernetes-with-kelsey-hightower/

**Below, please give your permission to download the audio clip and the transcript from the Screaming in the Cloud podcast mentioned above.**

In [ ]:
#@title Execute cell to accept downloading of podcast and its transcript
allow_download_of_podcast = "Don't Accept" #@param ["Don't Accept", "Accept"]


In [ ]:
# Download the podcast
import os

if not os.path.exists("data"):
  os.makedirs("data")
  
if allow_download_of_podcast != "Accept":
  raise RuntimeError("Request to download the podcast has been stopped as the user has not accepted downloading of audio")

if not os.path.exists("data/raw_audio.mp3"):
  !wget https://cdn.transistor.fm/file/transistor/m/shows/1494/62fad3e7a01de5beca8289352fe3a7bf.mp3 -O "data/raw_audio.mp3" -P data 

## Preprocess Audio

We now have the raw audio file (in mp3 format) from the podcast. To make this audio file compatible with the model (monochannel, 16 KHz audio), we will use FFMPEG to preprocess this file.

In [ ]:
import sys
import glob 
import subprocess

def transcode(input_dir, output_format, sample_rate, skip, duration):
    files = glob.glob(os.path.join(input_dir, "*.*"))

    # Filter out additional directories
    files = [f for f in files if not os.path.isdir(f)]

    output_dir = os.path.join(input_dir, "processed")

    if not os.path.exists(output_dir):
        print(f"Output directory {output_dir} does not exist, creating ...")
        os.makedirs(output_dir)

    for filepath in files:
        output_filename = os.path.basename(filepath)
        output_filename = os.path.splitext(output_filename)[0]

        output_filename = f"{output_filename}_processed.{output_format}"

        args = [
            'ffmpeg',
            '-i',
            str(filepath),
            '-ar',
            str(sample_rate),
            '-ac',
            str(1),
            '-y'
        ]

        if skip is not None:
            args.extend(['-ss', str(skip)])

        if duration is not None:
            args.extend(['-to', str(duration)])

        args.append(os.path.join(output_dir, output_filename))
        command = " ".join(args)
        !{command}

    print("\n")
    print(f"Finished trancoding {len(files)} audio files")


In [ ]:
transcode(
        input_dir="./data/",
        output_format="wav",
        sample_rate=16000,
        skip=None,
        duration=None,
    )

## Prepare transcript

The original transcript section is provided below. We then preprocess the text in the transcript to construct a "ground truth" text corpus for this podcast.

In [ ]:
#@title Execute cell to initialize raw transcript!

if allow_download_of_podcast != "Accept":
  raise RuntimeError("Request to download the podcast has been stopped as the user has not accepted downloading of the transcript")

raw_transcript = """
Announcer: Hello and welcome to Screaming in the Cloud, with your host Cloud economist Corey Quinn. This weekly show features conversations with people doing interesting work in the world of Cloud, thoughtful commentary on the state of the technical world, and ridiculous titles for which Corey refuses to apologize. This is Screaming in the Cloud.


Corey: This episode is sponsored by CloudZero. CloudZero wants you to know a few things. One: they realize it's not that original to put a random noun after the word cloud to name your company. Yet somehow they did it anyway. Two: most cost management solutions weren't designed for engineering teams, and three, spending $50,000 on something by accident would be bananas in virtually any other industry. That's why CloudZero built a product that helped engineers correlate their costs with the engineering activity that caused it, so you can quickly detect anomalous span, investigate the source of any cost, and slice up your AWS costs by the metrics that matter to you. Go to cloudzero.com to kick off a free trial that's cloud Z E R O.com and my thanks to them for sponsoring this episode.


Corey: This episode is brought to you by DigitalOcean, the cloud provider that makes it easy for startups to deploy and scale modern web applications with, and this is important to me: No billing surprises. With simple, predictable pricing that's flat across 12 global data center regions, and a UX developers around the world love, you can control your cloud infrastructure costs and have more time for your team to focus on growing your business. See what businesses are building on DigitalOcean and get started for free at do.co/screaming. That's D-O-Dot-C-O-slash-screaming, and my thanks to DigitalOcean for their continuing support of this ridiculous podcast.


Corey: Welcome to Screaming in the Cloud, I'm Corey Quinn. I'm joined this week by Kelsey Hightower, who claims to be a principal developer advocate at Google, but based upon various keynotes I've seen him in, he basically gets on stage and plays video games like Tetris in front of large audiences. So I assume he is somehow involved with e-sports. Kelsey, welcome to the show.


Kelsey: You've outed me. Most people didn't know that I am a full-time e-sports Tetris champion at home. And the technology thing is just a side gig.


Corey: Exactly. It's one of those things you do just to keep the lights on, like you're waiting to get discovered, but in the meantime, you're waiting table. Same type of thing. Some people wait tables you more or less a sling Kubernetes, for lack of a better term.


Kelsey: Yes.


Corey: So let's dive right into this. You've been a strong proponent for a long time of Kubernetes and all of its intricacies and all the power that it unlocks and I've been pretty much the exact opposite of that, as far as saying it tends to be over complicated, that it's hype-driven and a whole bunch of other, shall we say criticisms that are sometimes bounded in reality and sometimes just because I think it'll be funny when I put them on Twitter. Where do you stand on the state of Kubernetes in 2020?


Kelsey: So, I want to make sure it's clear what I do. Because when I started talking about Kubernetes, I was not working at Google. I was actually working at CoreOS where we had a competitor Kubernetes called Fleet. And Kubernetes coming out kind of put this like fork in our roadmap, like where do we go from here? What people saw me doing with Kubernetes was basically learning in public. Like I was really excited about the technology because it's attempting to solve a very complex thing. I think most people will agree building a distributed system is what cloud providers typically do, right? With VMs and hypervisors. Those are very big, complex distributed systems. And before Kubernetes came out, the closest I'd gotten to a distributed system before working at CoreOS was just reading the various white papers on the subject and hearing stories about how Google has systems like Borg tools, like Mesa was being used by some of the largest hyperscalers in the world, but I was never going to have the chance to ever touch one of those unless I would go work at one of those companies.


So when Kubernetes came out and the fact that it was open source and I could read the code to understand how it was implemented, to understand how schedulers actually work and then bonus points for being able to contribute to it. Those early years, what you saw me doing was just being so excited about systems that I attended to build on my own, becoming this new thing just like Linux came up. So I kind of agree with you that a lot of people look at it as a more of a hype thing. They're looking at it regardless of their own needs, regardless of understanding how it works and what problems is trying to solve that. My stance on it, it's a really, really cool tool for the level that it operates in, and in order for it to be successful, people can't know that it's there.


Corey: And I think that might be where part of my disconnect from Kubernetes comes into play. I have a background in ops, more or less, the grumpy Unix sysadmin because it's not like there's a second kind of Unix sysadmin you're ever going to encounter. Where everything in development works in theory, but in practice things pan out a little differently. I always joke that ops is the difference between theory and practice. In theory, devs can do everything and there's no ops needed. In practice, well it's been a burgeoning career for a while. The challenge with this is Kubernetes at times exposes certain levels of abstraction that, sorry certain levels of detail that generally people would not want to have to think about or deal with, while papering over other things with other layers of abstraction on top of it. That obscure, valuable troubleshooting information from a running something in an operational context. It absolutely is a fascinating piece of technology, but it feels today like it is overly complicated for the use a lot of people are attempting to put it to. Is that a fair criticism from where you sit?


Kelsey: So I think the reason why it's a fair criticism is because there are people attempting to run their own Kubernetes cluster, right? So when we think about the cloud, unless you're in OpenStack land, but for the people who look at the cloud and you say, "Wow, this is much easier." There's an API for creating virtual machines and I don't see the distributed state store that's keeping all of that together. I don't see the farm of hypervisors. So we don't necessarily think about the inherent complexity into a system like that, because we just get to use it. So on one end, if you're just a user of a Kubernetes cluster, maybe using something fully managed or you have an ops team that's taking care of everything, your interface of the system becomes this Kubernetes configuration language where you say, "Give me a load balancer, give me three copies of this container running." And if we do it well, then you'd think it's a fairly easy system to deal with because you say, "kubectl, apply," and things seem to start running.


Just like in the cloud where you say, "AWS create this VM, or G cloud compute instance, create." You just submit API calls and things happen. I think the fact that Kubernetes is very transparent to most people is, now you can see the complexity, right? Imagine everyone driving with the hood off the car. You'd be looking at a lot of moving things, but we have hoods on cars to hide the complexity and all we expose is the steering wheel and the pedals. That car is super complex but we don't see it. So therefore we don't attribute as complexity to the driving experience.


Corey: This to some extent feels it's on the same axis as serverless, with just a different level of abstraction piled onto it. And while I am a large proponent of serverless, I think it's fantastic for a lot of Greenfield projects. The constraints inherent to the model mean that it is almost completely non-tenable for a tremendous number of existing workloads. Some developers like to call it legacy, but when I hear the term legacy I hear, "it makes actual money." So just treating it as, "Oh, it's a science experiment we can throw into a new environment, spend a bunch of time rewriting it for minimal gains," is just not going to happen as companies undergo digital transformations, if you'll pardon the term.


Kelsey: Yeah, so I think you're right. So let's take Amazon's Lambda for example, it's a very opinionated high-level platform that assumes you're going to build apps a certain way. And if that's you, look, go for it. Now, one or two levels below that there is this distributed system. Kubernetes decided to play in that space because everyone that's building other platforms needs a place to start. The analogy I like to think of is like in the mobile space, iOS and Android deal with the complexities of managing multiple applications on a mobile device, security aspects, app stores, that kind of thing. And then you as a developer, you build your thing on top of those platforms and APIs and frameworks. Now, it's debatable, someone would say, "Why do we even need an open-source implementation of such a complex system? Why not just everyone moved to the cloud?" And then everyone that's not in a cloud on-premise gets left behind.


But typically that's not how open source typically works, right? The reason why we have Linux, the precursor to the cloud is because someone looked at the big proprietary Unix systems and decided to re-implement them in a way that anyone could run those systems. So when you look at Kubernetes, you have to look at it from that lens. It's the ability to democratize these platform layers in a way that other people can innovate on top. That doesn't necessarily mean that everyone needs to start with Kubernetes, just like not everyone needs to start with the Linux server, but it's there for you to build the next thing on top of, if that's the route you want to go.


Corey: It's been almost a year now since I made an original tweet about this, that in five years, no one will care about Kubernetes. So now I guess I have four years running on that clock and that attracted a bit of, shall we say controversy. There were people who thought that I meant that it was going to be a flash in the pan and it would dry up and blow away. But my impression of it is that in, well four years now, it will have become more or less system D for the data center, in that there's a bunch of complexity under the hood. It does a bunch of things. No-one sensible wants to spend all their time mucking around with it in most companies. But it's not something that people have to think about in an ongoing basis the way it feels like we do today.


Kelsey: Yeah, I mean to me, I kind of see this as the natural evolution, right? It's new, it gets a lot of attention and kind of the assumption you make in that statement is there's something better that should be able to arise, giving that checkpoint. If this is what people think is hot, within five years surely we should see something else that can be deserving of that attention, right? Docker comes out and almost four or five years later you have Kubernetes. So it's obvious that there should be a progression here that steals some of the attention away from Kubernetes, but I think where it's so new, right? It's only five years in, Linux is like over 20 years old now at this point, and it's still top of mind for a lot of people, right? Microsoft is still porting a lot of Windows only things into Linux, so we still discuss the differences between Windows and Linux.


The idea that the cloud, for the most part, is driven by Linux virtual machines, that I think the majority of workloads run on virtual machines still to this day, so it's still front and center, especially if you're a system administrator managing BDMs, right? You're dealing with tools that target Linux, you know the Cisco interface and you're thinking about how to secure it and lock it down. Kubernetes is just at the very first part of that life cycle where it's new. We're all interested in even what it is and how it works, and now we're starting to move into that next phase, which is the distro phase. Like in Linux, you had Red Hat, Slackware, Ubuntu, special purpose distros.


Some will consider Android a special purpose distribution of Linux for mobile devices. And now that we're in this distro phase, that's going to go on for another 5 to 10 years where people start to align themselves around, maybe it's OpenShift, maybe it's GKE, maybe it's Fargate for EKS. These are now distributions built on top of Kubernetes that start to add a little bit more opinionation about how Kubernetes should be pushed together. And then we'll enter another phase where you'll build a platform on top of Kubernetes, but it won't be worth mentioning that Kubernetes is underneath because people will be more interested on the thing above.


Corey: I think we're already seeing that now, in terms of people no longer really care that much what operating system they're running, let alone with distribution of that operating system. The things that you have to care about slip below the surface of awareness and we've seen this for a long time now. Originally to install a web server, it wound up taking a few days and an intimate knowledge of GCC compiler flags, then RPM or D package and then yum on top of that, then ensure installed, once we had configuration management that was halfway decent.


Then Docker run, whatever it is. And today feels like it's with serverless technologies being what they are, it's effectively a push a file to S3 or it's equivalent somewhere else and you're done. The things that people have to be aware of and the barrier to entry continually lowers. The downside to that of course, is that things that people specialize in today and effectively make very lucrative careers out of are going to be not front and center in 5 to 10 years the way that they are today. And that's always been the way of technology. It's a treadmill to some extent.


Kelsey: And on the flip side of that, look at all of the new jobs that are centered around these cloud-native technologies, right? So you know, we're just going to make up some numbers here, imagine if there were only 10,000 jobs around just Linux system administration. Now when you look at this whole Kubernetes landscape where people are saying we can actually do a better job with metrics and monitoring. Observability is now a thing culturally that people assume you should have, because you're dealing with these distributed systems. The ability to start thinking about multi-regional deployments when I think that would've been infeasible with the previous tools or you'd have to build all those tools yourself. So I think now we're starting to see a lot more opportunities, where instead of 10,000 people, maybe you need 20,000 people because now you have the tools necessary to tackle bigger projects where you didn't see that before.


Corey: That's what's going to be really neat to see. But the challenge is always to people who are steeped in existing technologies. What does this mean for them? I mean I spent a lot of time early in my career fighting against cloud because I thought that it was taking away a cornerstone of my identity. I was a large scale Unix administrator, specifically focusing on email. Well, it turns out that there aren't nearly as many companies that need to have that particular skill set in house as it did 10 years ago. And what we're seeing now is this sort of forced evolution of people's skillsets or they hunker down on a particular area of technology or particular application to try and make a bet that they can ride that out until retirement. It's challenging, but at some point it seems that some folks like to stop learning, and I don't fully pretend to understand that. I'm sure I will someday where, "No, at this point technology come far enough. We're just going to stop here, and anything after this is garbage." I hope not, but I can see a world in which that happens.


Kelsey: Yeah, and I also think one thing that we don't talk a lot about in the Kubernetes community, is that Kubernetes makes hyper-specialization worth doing because now you start to have a clear separation from concerns. Now the OS can be hyperfocused on security system calls and not necessarily packaging every programming language under the sun into a single distribution. So we can kind of move part of that layer out of the core OS and start to just think about the OS being a security boundary where we try to lock things down. And for some people that play at that layer, they have a lot of work ahead of them in locking down these system calls, improving the idea of containerization, whether that's something like Firecracker or some of the work that you see VMware doing, that's going to be a whole class of hyper-specialization. And the reason why they're going to be able to focus now is because we're starting to move into a world, whether that's serverless or the Kubernetes API.


We're saying we should deploy applications that don't target machines. I mean just that step alone is going to allow for so much specialization at the various layers because even on the networking front, which arguably has been a specialization up until this point, can truly specialize because now the IP assignments, how networking fits together, has also abstracted a way one more step where you're not asking for interfaces or binding to a specific port or playing with port mappings. You can now let the platform do that. So I think for some of the people who may be not as interested as moving up the stack, they need to be aware that the number of people we need being hyper-specialized at Linux administration will definitely shrink. And a lot of that work will move up the stack, whether that's Kubernetes or managing a serverless deployment and all the configuration that goes with that. But if you are a Linux, like that is your bread and butter, I think there's going to be an opportunity to go super deep, but you may have to expand into things like security and not just things like configuration management.


Corey: Let's call it the unfulfilled promise of Kubernetes. On paper, I love what it hints at being possible. Namely, if I build something that runs well on top of Kubernetes than we truly have a write once, run anywhere type of environment. Stop me if you've heard that one before, 50,000 times in our industry... or history. But in practice, as has happened before, it seems like it tends to fall down for one reason or another. Now, Amazon is famous because for many reasons, but the one that I like to pick on them for is, you can't say the word multi-cloud at their events. Right. That'll change people's perspective, good job. The people tend to see multi-cloud are a couple of different lenses.


I've been rather anti multi-cloud from the perspective of the idea that you're setting out day one to build an application with the idea that it can be run on top of any cloud provider, or even on-premises if that's what you want to do, is generally not the way to proceed. You wind up having to make certain trade-offs along the way, you have to rebuild anything that isn't consistent between those providers, and it slows you down. Kubernetes on the other hand hints at if it works and fulfills this promise, you can suddenly abstract an awful lot beyond that and just write generic applications that can run anywhere. Where do you stand on the whole multi-cloud topic?


Kelsey: So I think we have to make sure we talk about the different layers that are kind of ready for this thing. So for example, like multi-cloud networking, we just call that networking, right? What's the IP address over there? I can just hit it. So we don't make a big deal about multi-cloud networking. Now there's an area where people say, how do I configure the various cloud providers? And I think the healthy way to think about this is, in your own data centers, right, so we know a lot of people have investments on-premises. Now, if you were to take the mindset that you only need one provider, then you would try to buy everything from HP, right? You would buy HP store's devices, you buy HP racks, power. Maybe HP doesn't sell air conditioners. So you're going to have to buy an air conditioner from a vendor who specializes in making air conditioners, hopefully for a data center and not your house.


So now you've entered this world where one vendor does it make every single piece that you need. Now in the data center, we don't say, "Oh, I am multi-vendor in my data center." Typically, you just buy the switches that you need, you buy the power racks that you need, you buy the ethernet cables that you need, and they have common interfaces that allow them to connect together and they typically have different configuration languages and methods for configuring those components. The cloud on the other hand also represents the same kind of opportunity. There are some people who really love DynamoDB and S3, but then they may prefer something like BigQuery to analyze the data that they're uploading into S3. Now, if this was a data center, you would just buy all three of those things and put them in the same rack and call it good.


But the cloud presents this other challenge. How do you authenticate to those systems? And then there's usually this additional networking costs, egress or ingress charges that make it prohibitive to say, "I want to use two different products from two different vendors." And I think that's-


Corey: ...winds up causing serious problems.


Kelsey: Yes, so that data gravity, the associated cost becomes a little bit more in your face. Whereas, in a data center you kind of feel that the cost has already been paid. I already have a network switch with enough bandwidth, I have an extra port on my switch to plug this thing in and they're all standard interfaces. Why not? So I think the multi-cloud gets lost in the chew problem, which is the barrier to entry of leveraging things across two different providers because of networking and configuration practices.


Corey: That's often the challenge, I think, that people get bogged down in. On an earlier episode of this show we had Mitchell Hashimoto on, and his entire theory around using Terraform to wind up configuring various bits of infrastructure, was not the idea of workload portability because that feels like the windmill we all keep tilting at and failing to hit. But instead the idea of workflow portability, where different things can wind up being interacted with in the same way. So if this one division is on one cloud provider, the others are on something else, then you at least can have some points of consistency in how you interact with those things. And in the event that you do need to move, you don't have to effectively redo all of your CICD process, all of your tooling, et cetera. And I thought that there was something compelling about that argument.


Kelsey: And that's actually what Kubernetes does for a lot of people. For Kubernetes, if you think about it, when we start to talk about workflow consistency, if you want to deploy an application, queue CTL, apply, some config, you want the application to have a load balancer in front of it. Regardless of the cloud provider, because Kubernetes has an extension point we call the cloud provider. And that's where Amazon, Azure, Google Cloud, we do all the heavy lifting of mapping the high-level ingress object that specifies, "I want a load balancer, maybe a few options," to the actual implementation detail. So maybe you don't have to use four or five different tools and that's where that kind of workload portability comes from. Like if you think about Linux, right? It has a set of system calls, for the most part, even if you're using a different distro at this point, Red Hat or Amazon Linux or Google's container optimized Linux.


If I build a Go binary on my laptop, I can SCP it to any of those Linux machines and it's going to probably run. So you could call that multi-cloud, but that doesn't make a lot of sense because it's just because of the way Linux works. Kubernetes does something very similar because it sits right on top of Linux, so you get the portability just from the previous example and then you get the other portability and workload, like you just stated, where I'm calling kubectl apply, and I'm using the same workflow to get resources spun up on the various cloud providers. Even if that configuration isn't one-to-one identical.


Corey: This episode is sponsored in part by DataStax. The NoSQL event of the year is DataStax Accelerate in San Diego this May from the 11th through the 13th. I've given a talk previously called the myth of multi-cloud, and it's time for me to revisit that with... A sequel! Which is funny given that it's a NoSQL conference, but there you have it. To learn more, visit datastax.com that's D-A-T-A-S-T-A-X.com and I hope to see you in San Diego. This May.


Corey: One thing I'm curious about is you wind up walking through the world and seeing companies adopting Kubernetes in different ways. How are you finding the adoption of Kubernetes is looking like inside of big E enterprise style companies? I don't have as much insight into those environments as I probably should. That's sort of a focus area for the next year for me. But in startups, it seems that it's either someone goes in and rolls it out and suddenly it's fantastic, or they avoid it entirely and do something serverless. In large enterprises, I see a lot of Kubernetes and a lot of Kubernetes stories coming out of it, but what isn't usually told is, what's the tipping point where they say, "Yeah, let's try this." Or, "Here's the problem we're trying to solve for. Let's chase it."


Kelsey: What I see is enterprises buy everything. If you're big enough and you have a big enough IT budget, most enterprises have a POC of everything that's for sale, period. There's some team in some pocket, maybe they came through via acquisition. Maybe they live in a different state. Maybe it's just a new project that came out. And what you tend to see, at least from my experiences, if I walk into a typical enterprise, they may tell me something like, "Hey, we have a POC, a Pivotal Cloud Foundry, OpenShift, and we want some of that new thing that we just saw from you guys. How do we get a POC going?" So there's always this appetite to evaluate what's for sale, right? So, that's one case. There's another case where, when you start to think about an enterprise there's a big range of skillsets. Sometimes I'll go to some companies like, "Oh, my insurance is through that company, and there's ex-Googlers that work there." They used to work on things like Borg, or something else, and they kind of know how these systems work.


And they have a slightly better edge at evaluating whether Kubernetes is any good for the problem at hand. And you'll see them bring it in. Now that same company, I could drive over to the other campus, maybe it's five miles away and that team doesn't even know what Kubernetes is. And for them, they're going to be chugging along with what they're currently doing. So then the challenge becomes if Kubernetes is a great fit, how wide of a fit it isn't? How many teams at that company should be using it? So what I'm currently seeing as there are some enterprises that have found a way to make Kubernetes the place where they do a lot of new work, because that makes sense. A lot of enterprises to my surprise though, are actually stepping back and saying, "You know what? We've been stitching together our own platform for the last five years. We had the Netflix stack, we got some Spring Boot, we got Console, we got Vault, we got Docker. And now this whole thing is getting a little more fragile because we're doing all of this glue code."


Kubernetes, We've been trying to build our own Kubernetes and now that we know what it is and we know what it isn't, we know that we can probably get rid of this kind of bespoke stack ourselves and just because of the ecosystem, right? If I go to HashiCorp's website, I would probably find the word Kubernetes as much as I find the word Nomad on their site because they've made things like Console and Vault become first-class offerings inside of the world of Kubernetes. So I think it's that momentum that you see across even People Oracle, Juniper, Palo Alto Networks, they're all have seem to have a Kubernetes story. And this is why you start to see the enterprise able to adopt it because it's so much in their face and it's where the ecosystem is going.


Corey: It feels like a lot of the excitement and the promise and even the same problems that Kubernetes is aimed at today, could have just as easily been talked about half a decade ago in the context of OpenStack. And for better or worse, OpenStack is nowhere near where it once was. It would felt like it had such promise and such potential and when it didn't pan out, that left a lot of people feeling relatively sad, burnt out, depressed, et cetera. And I'm seeing a lot of parallels today, at least between what was said about OpenStack and what was said about Kubernetes. How do you see those two diverging?


Kelsey: I will tell you the big difference that I saw, personally. Just for my personal journey outside of Google, just having that option. And I remember I was working at a company and we were like, "We're going to roll our own OpenStack. We're going to buy a free BSD box and make it a file server. We're going all open sources," like do whatever you want to do. And that was just having so many issues in terms of first-class integrations, education, people with the skills to even do that. And I was like, "You know what, let's just cut the check for VMware." We want virtualization. VMware, for the cost and when it does, it's good enough. Or we can just actually use a cloud provider. That space in many ways was a purely solved problem. Now, let's fast forward to Kubernetes, and also when you get OpenStack finished, you're just back where you started.


You got a bunch of VMs and now you've got to go figure out how to build the real platform that people want to use because no one just wants a VM. If you think Kubernetes is low level, just having OpenStack, even OpenStack was perfect. You're still at square one for the most part. Maybe you can just say, "Now I'm paying a little less money for my stack in terms of software licensing costs," but from an extraction and automation and API standpoint, I don't think OpenStack moved the needle in that regard. Now in the Kubernetes world, it's solving a huge gap.


Lots of people have virtual machine sprawl than they had Docker sprawl, and when you bring in this thing by Kubernetes, it says, "You know what? Let's reign all of that in. Let's build some first-class abstractions, assuming that the layer below us is a solved problem." You got to remember when Kubernetes came out, it wasn't trying to replace the hypervisor, it assumed it was there. It also assumed that the hypervisor had APIs for creating virtual machines and attaching disc and creating load balancers, so Kubernetes came out as a complementary technology, not one looking to replace. And I think that's why it was able to stick because it solved a problem at another layer where there was not a lot of competition.


Corey: I think a more cynical take, at least one of the ones that I've heard articulated and I tend to agree with, was that OpenStack originally seemed super awesome because there were a lot of interesting people behind it, fascinating organizations, but then you wound up looking through the backers of the foundation behind it and the rest. And there were something like 500 companies behind it, an awful lot of them were these giant organizations that ... they were big e-corporate IT enterprise software vendors, and you take a look at that, I'm not going to name anyone because at that point, oh will we get letters.


But at that point, you start seeing so many of the patterns being worked into it that it almost feels like it has to collapse under its own weight. I don't, for better or worse, get the sense that Kubernetes is succumbing to the same thing, despite the CNCF having an awful lot of those same backers behind it and as far as I can tell, significantly more money, they seem to have all the money to throw at these sorts of things. So I'm wondering how Kubernetes has managed to effectively sidestep I guess the open-source miasma that OpenStack didn't quite manage to avoid.


Kelsey: Kubernetes gained its own identity before the foundation existed. Its purpose, if you think back from the Borg paper almost eight years prior, maybe even 10 years prior. It defined this problem really, really well. I think Mesos came out and also had a slightly different take on this problem. And you could just see at that time there was a real need, you had choices between Docker Swarm, Nomad. It seems like everybody was trying to fill in this gap because, across most verticals or industries, this was a true problem worth solving. What Kubernetes did was played in the exact same sandbox, but it kind of got put out with experience. It's not like, "Oh, let's just copy this thing that already exists, but let's just make it open."


And in that case, you don't really have your own identity. It's you versus Amazon, in the case of OpenStack, it's you versus VMware. And that's just really a hard place to be in because you don't have an identity that stands alone. Kubernetes itself had an identity that stood alone. It comes from this experience of running a system like this. It comes from research and white papers. It comes after previous attempts at solving this problem. So we agree that this problem needs to be solved. We know what layer it needs to be solved at. We just didn't get it right yet, so Kubernetes didn't necessarily try to get it right.


It tried to start with only the primitives necessary to focus on the problem at hand. Now to your point, the extension interface of Kubernetes is what keeps it small. Years ago I remember plenty of meetings where we all got in rooms and said, "This thing is done." It doesn't need to be a PaaS. It doesn't need to compete with serverless platforms. The core of Kubernetes, like Linux, is largely done. Here's the core objects, and we're going to make a very great extension interface. We're going to make one for the container run time level so that way people can swap that out if they really want to, and we're going to do one that makes other APIs as first-class as ones we have, and we don't need to try to boil the ocean in every Kubernetes release. Everyone else has the ability to deploy extensions just like Linux, and I think that's why we're avoiding some of this tension in the vendor world because you don't have to change the core to get something that feels like a native part of Kubernetes.


Corey: What do you think is currently being the most misinterpreted or misunderstood aspect of Kubernetes in the ecosystem?


Kelsey: I think the biggest thing that's misunderstood is what Kubernetes actually is. And the thing that made it click for me, especially when I was writing the tutorial Kubernetes The Hard Way. I had to sit down and ask myself, "Where do you start trying to learn what Kubernetes is?" So I start with the database, right? The configuration store isn't Postgres, it isn't MySQL, it's Etcd. Why? Because we're not trying to be this generic data stores platform. We just need to store configuration data. Great. Now, do we let all the components talk to Etcd? No. We have this API server and between the API server and the chosen data store, that's essentially what Kubernetes is. You can stop there. At that point, you have a valid Kubernetes cluster and it can understand a few things. Like I can say, using the Kubernetes command-line tool, create this configuration map that stores configuration data and I can read it back.


Great. Now I can't do a lot of things that are interesting with that. Maybe I just use it as a configuration store, but then if I want to build a container platform, I can install the Kubernetes kubelet agent on a bunch of machines and have it talk to the API server looking for other objects you add in the scheduler, all the other components. So what that means is that Kubernetes most important component is its API because that's how the whole system is built. It's actually a very simple system when you think about just those two components in isolation. If you want a container management tool that you need a scheduler, controller, manager, cloud provider integrations, and now you have a container tool. But let's say you want a service mesh platform. Well in a service mesh you have a data plane that can be Nginx or Envoy and that's going to handle routing traffic. And you need a control plane. That's going to be something that takes in configuration and it uses that to configure all the things in a data plane.


Well, guess what? Kubernetes is 90% there in terms of a control plane, with just those two components, the API server, and the data store. So now when you want to build control planes, if you start with the Kubernetes API, we call it the API machinery, you're going to be 95% there. And then what do you get? You get a distributed system that can handle kind of failures on the back end, thanks to Etcd. You're going to get our backs or you can have permission on top of your schemas, and there's a built-in framework, we call it custom resource definitions that allows you to articulate a schema and then your own control loops provide meaning to that schema. And once you do those two things, you can build any platform you want. And I think that's one thing that it takes a while for people to understand that part of Kubernetes, that the thing we talk about today, for the most part, is just the first system that we built on top of this.


Corey: I think that's a very far-reaching story with implications that I'm not entirely sure I am able to wrap my head around. I hope to see it, I really do. I mean you mentioned about writing Learn Kubernetes the Hard Way and your tutorial, which I'll link to in the show notes. I mean my, of course, sarcastic response to that recently was to register the domain Kubernetes the Easy Way and just re-pointed to Amazon's ECS, which is in no way shape or form Kubernetes and basically has the effect of irritating absolutely everyone as is my typical pattern of behavior on Twitter. But I have been meaning to dive into Kubernetes on a deeper level and the stuff that you've written, not just the online tutorial, both the books have always been my first port of call when it comes to that. The hard part, of course, is there's just never enough hours in the day.


Kelsey: And one thing that I think about too is like the web. We have the internet, there's webpages, there's web browsers. Web Browsers talk to web servers over HTTP. There's verbs, there's bodies, there's headers. And if you look at it, that's like a very big complex system. If I were to extract out the protocol pieces, this concept of HTTP verbs, get, put, post and delete, this idea that I can put stuff in a body and I can give it headers to give it other meaning and semantics. If I just take those pieces, I can bill restful API's.


Hell, I can even bill graph QL and those are just different systems built on the same API machinery that we call the internet or the web today. But you have to really dig into the details and pull that part out and you can build all kind of other platforms and I think that's what Kubernetes is. It's going to probably take people a little while longer to see that piece, but it's hidden in there and that's that piece that's going to be, like you said, it's going to probably be the foundation for building more control planes. And when people build control planes, I think if you think about it, maybe Fargate for EKS represents another control plane for making a serverless platform that takes to Kubernetes API, even though the implementation isn't what you find on GitHub.


Corey: That's the truth. Whenever you see something as broadly adopted as Kubernetes, there's always the question of, "Okay, there's an awful lot of blog posts." Getting started to it, learn it in 10 minutes, I mean at some point, I'm sure there are some people still convince Kubernetes is, in fact, a breakfast cereal based upon what some of the stuff the CNCF has gotten up to. I wouldn't necessarily bet against it socks today, breakfast cereal tomorrow. But it's hard to find a decent level of quality, finding the certain quality bar of a trusted source to get started with is important. Some people believe in the hero's journey, story of a narrative building.


I always prefer to go with the morons journey because I'm the moron. I touch technologies, I have no idea what they do and figure it out and go careening into edge and corner cases constantly. And by the end of it I have something that vaguely sort of works and my understanding's improved. But I've gone down so many terrible paths just by picking a bad point to get started. So everyone I've talked to who's actually good at things has pointed to your work in this space as being something that is authoritative and largely correct and given some of these people, that's high praise.


Kelsey: Awesome. I'm going to put that on my next performance review as evidence of my success and impact.


Corey: Absolutely. Grouchy people say, "It's all right," you know, for the right people that counts. If people want to learn more about what you're up to and see what you have to say, where can they find you?


Kelsey: I aggregate most of outward interactions on Twitter, so I'm @KelseyHightower and my DMs are open, so I'm happy to field any questions and I attempt to answer as many as I can.


Corey: Excellent. Thank you so much for taking the time to speak with me today. I appreciate it.


Kelsey: Awesome. I was happy to be here.


Corey: Kelsey Hightower, Principal Developer Advocate at Google. I'm Corey Quinn. This is Screaming in the Cloud. If you've enjoyed this podcast, please leave a five-star review on Apple podcasts. If you've hated this podcast, please leave a five-star review on Apple podcasts and then leave a funny comment. Thanks.


Announcer: This has been this week's episode of Screaming in the Cloud. You can also find more Core at screaminginthecloud.com or wherever fine snark is sold.


Announcer: This has been a HumblePod production. Stay humble.
"""

In [ ]:
# write out the transcript into a file
if not os.path.exists("transcripts"):
  os.makedirs("transcripts")

with open("transcripts/raw_transcript.txt", 'w') as f:
  f.write(raw_transcript)
  f.write("\n")

# Text Preprocessing pipeline

Here, we create a basic preprocessing pipeline for the transcription to get it into a format that can easily be compared against the model's transcription. We won't focus on efficiency here, instead opting to implement just a simple pipeline processor that offers some flexibility.

In [ ]:
import re
from tqdm import tqdm
from typing import List, Optional

### Preprocessing tasks

First we create a basic unit of work in the pipeline - aka a "task" that will be accept some input and will emit some output. 

It will also maintain a dictionary of metadata so that subsequent tasks can monitor / use the previous steps' metadata information (if needed).

In [ ]:
class ProcessingTask:
    def __init__(self):
        self.metadata = {}

    def __call__(self, *args):
        raise NotImplementedError()

### Text Preprocessor

Next we create the actual processor of multiple tasks. We will impose a few restrictions - the input must be args (no kwargs), output must also be args (no dicts). 

Each task can also read/update the global metadata registry which will be updated with the metadata of all the previous tasks.

In [ ]:
class TextPreprocessor:
    def __init__(self, filepath):
        self.filepath = filepath
        self.metadata = {}

    def process(self, tasks: List[ProcessingTask]):
        # read the text file in its entirety
        with open(self.filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        print(f"Loaded {len(lines)} lines of text into memory ...")
        assert len(tasks) > 0

        # Prepare the processing pipeline
        processed_outputs = [lines]
        metadata = self.metadata
        metadata['task_pipeline'] = []  # keep track of order of tasks executed

        for task in tasks:
            print(f"Performing task : {task.__class__.__name__}")
            metadata['task_pipeline'].append(task.__class__.__name__)
            task.metadata.update(metadata)  # update the global metadata with the previous tasks metadata

            processed_outputs = task(*processed_outputs)  # process the tasks
            metadata = task.metadata

            # if output was a single value, pack it into a tuple
            if type(processed_outputs) != tuple:
                processed_outputs = (processed_outputs,)

        self.metadata = metadata

        return processed_outputs

### Generic Processing Tasks

Now that we have the basic task format and its executer, let's create some basic text preprocessing tasks.

In [ ]:
class TrimNewLinesTask(ProcessingTask):
    """ Trim all new lines from the text """
    def __call__(self, texts, **kwargs):
        for idx in tqdm(range(len(texts)), desc=self.__class__.__name__, total=len(texts)):
            texts[idx] = texts[idx].replace("\n", "")

        return texts

In [ ]:
class IndexBlankLinesTask(ProcessingTask):
    """ Get the index of blank lines within the text (after replacing newlines) """
    def __init__(self):
        super(IndexBlankLinesTask, self).__init__()
        self.metadata['blank_idx'] = []

    def __call__(self, texts, **kwargs):
        for idx in tqdm(range(len(texts)), desc=self.__class__.__name__, total=len(texts)):
            if len(texts[idx]) == 0:  # was just a new line
                self.metadata['blank_idx'].append(idx)

        return texts

In [ ]:
class LowerCaseTask(ProcessingTask):
    """ Lower case all of the text """
    def __call__(self, texts):
        for idx in tqdm(range(len(texts)), desc=self.__class__.__name__, total=len(texts)):
            texts[idx] = texts[idx].lower()

        return texts

In [ ]:
class SpecialCharactersTask(ProcessingTask):
    """ Replace a set of special characters with empty spaces """
    def __init__(self, special_characters: List[str], replacement: str = ""):
        super(SpecialCharactersTask, self).__init__()
        self.special_characters = special_characters
        self.replacement = replacement

    def __call__(self, texts):
        self.metadata['special_characters_idx'] = self.metadata.get('special_characters_idx', {})
        self.metadata['special_characters_replacement_tokens'] = self.metadata.get('special_characters_replacement_tokens', [])
        self.metadata['special_characters_replacement_tokens'].append(self.replacement)

        for idx in tqdm(range(len(texts)), desc=self.__class__.__name__, total=len(texts)):
            for special_char in self.special_characters:
                if special_char in texts[idx]:
                    texts[idx] = texts[idx].replace(special_char, self.replacement).strip()

                    if special_char in self.metadata['special_characters_idx']:
                        self.metadata['special_characters_idx'][special_char].append(idx)
                    else:
                        self.metadata['special_characters_idx'][special_char] = [idx]

        return texts

In [ ]:
class EmptyLineRemovalTask(ProcessingTask):
    """ Remove all empty lines from the text list """
    def __call__(self, texts):
        new_texts = []
        for idx in tqdm(range(len(texts)), desc=self.__class__.__name__, total=len(texts)):
            if len(texts[idx]) > 0:
                new_texts.append(texts[idx])

        return new_texts

In [ ]:
class MergeLinesTask(ProcessingTask):
    """ Merge all of the text with spaces """
    def __call__(self, texts):
        self.metadata['num_lines_merged'] = len(texts)

        texts = ' '.join(texts)
        return [texts]

### Specialized Preprocessing Tasks

The podcast requires more processing as compared to the generic tasks above. We define them below. 

The `SpeakerHeaderRemovalTask` will remove the mention of the speaker - (whether the speaker was Corey, Kelsey etc).

The `SeperateTokenTask` will take a unique word - say "screaminginthecloud" and replace it with spaces. While the original text is meant to represent an entity, the ASR model has no such notion and will probably transcribe it as separate words.

Finally, the `WebsiteTransformTask` will replace the ".com" in websites to "dot com" since the ASR model has no special characters.

In [ ]:
class SpeakerHeaderRemovalTask(ProcessingTask):
    """ Remove the speaker header information (represented as the first name of the entity - Announcer, Corey, Kelsey etc) """
    def __call__(self, texts):
        self.metadata['unique_speakers'] = {}

        for idx in tqdm(range(len(texts)), desc=self.__class__.__name__, total=len(texts)):
            text = texts[idx]
            if len(text) > 0:
                search = re.search(r"[a-zA-z]*:", text)
                if search:
                    speaker = search.group(0)
                    if speaker in self.metadata['unique_speakers']:
                        self.metadata['unique_speakers'][speaker].append(idx)
                    else:
                        self.metadata['unique_speakers'][speaker] = [idx]

                    texts[idx] = texts[idx].replace(speaker, "").strip()

        return texts

In [ ]:
class SeperateTokenTask(ProcessingTask):
    """ there are several mentions of "digital ocean" - but as it is an entity, it is transcribed as one word. Split it."""

    def __init__(self, token: str, split_token: str):
      super().__init__()
      self.token = token
      self.split_token = split_token

    def __call__(self, texts):
        self.metadata['separate_token'] = self.metadata.get('separate_token', {})
        self.metadata['separate_token'][self.token] = self.split_token

        for idx in tqdm(range(len(texts)), desc=self.__class__.__name__, total=len(texts)):
            if self.token in texts[idx]:
                texts[idx] = texts[idx].replace(self.token, self.split_token)

        return texts

In [ ]:
class WebsiteTransformTask(ProcessingTask):
    """ There are multiple references to *.com - replace . with "dot" as model does not produce special characters like period and comma """
    def __call__(self, texts):
        self.metadata[f'website_convert_idx'] = {}

        for idx in tqdm(range(len(texts)), desc=self.__class__.__name__, total=len(texts)):
            search = re.search(r"[a-zA-z-]*\.com", texts[idx])
            if search:
                matched_text = search.group(0)
                texts[idx] = texts[idx].replace(".", " dot").strip()

                if matched_text in self.metadata[f'website_convert_idx']:
                    self.metadata[f'website_convert_idx'][matched_text].append(idx)
                else:
                    self.metadata[f'website_convert_idx'][matched_text] = [idx]

        return texts

## Preprocess the transcript

Now that the pipeline is setup, let's execute it to clean up the text!

**NOTE**: These preprocessing steps were arbitrarily chosen and may/may not be valid corrections. Feel free to comment out the tasks list below to change the pipeline, or to add even more tasks !

In [ ]:
raw_transcript_filepath = "transcripts/raw_transcript.txt"

In [ ]:
tasks = [
        TrimNewLinesTask(),
        LowerCaseTask(),
        SpeakerHeaderRemovalTask(),
        WebsiteTransformTask(),
        SeperateTokenTask("digitalocean", "digital ocean"),
        SeperateTokenTask("cloudzero", "cloud zero"),
        SeperateTokenTask("screaminginthecloud", "screaming in the cloud"),
        SpecialCharactersTask(special_characters=[",", "%", "-", "?", "@", '"', ":", ".", "!"]),
        SpecialCharactersTask(special_characters=["$"], replacement=""),
        SpecialCharactersTask(special_characters=["/"], replacement=" slash "),
        IndexBlankLinesTask(),
        EmptyLineRemovalTask(),
        MergeLinesTask(),
    ]

processor = TextPreprocessor(raw_transcript_filepath)
result = processor.process(tasks)

# remove the tuple packing
result = result[0]
# the output text has been merged into a single text, so no need for the list anymore
result= result[0]  

In [ ]:
print("Metadata")
print("*" * 80)
for key in processor.metadata.keys():
  print(f"{key} :")
  print(processor.metadata[key])
  print()

---------

The resultant text is now just a long string of text with no special characters and new lines

In [ ]:
print("Result :")
print(result)

Write the normalized transcript into an output file

In [ ]:
if not os.path.exists("transcripts/normalized/"):
  os.makedirs("transcripts/normalized/")

normalized_transcript_path = "transcripts/normalized/ground_truth.txt"

with open(normalized_transcript_path, 'w', encoding='utf-8') as f:
    f.write(f"{result}")
    f.write("\n")

# Transcribe the processed audio file

Now that we have a "ground truth" text transcript we can compare against, let's actually transcribe the podcast with a model !

## Helper methods

We define a few helper methods to enable automatic mixed precision if it is available in the colab GPU (if a GPU is being used at all)

In [ ]:
import contextlib
import torch

In [ ]:
# Helper for torch amp autocast
if torch.cuda.is_available():
    autocast = torch.cuda.amp.autocast
else:
    @contextlib.contextmanager
    def autocast():
        print("AMP was not available, using FP32!")
        yield

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## Instantiate a model

We choose a small model - Citrinet 256 - since it offers good transcription accuracy but is just 10 M parameters.

**Feel free to change to the medium and larger sized models !**

 - small = "stt_en_citrinet_256" (9.8 M parameters)
 - medium = "stt_en_citrinet_512" (38 M parameters)
 - large = "stt_en_citrinet_1024" (142 M parameters)

In [ ]:
model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained("stt_en_citrinet_256", map_location=device)

In [ ]:
model = model.to(device)

## Transcribe audio

Here, we simply call the model's "transcribe()" method, which does offline transcription of a provided list of audio clips.

In [ ]:
%%time

audio_path = "data/processed/raw_audio_processed.wav"
transcribed_filepath = f"transcripts/normalized/transcribed_speech.txt"

if os.path.exists(transcribed_filepath):
  print(f"File already exists, delete {transcribed_filepath} manually before re-transcribing the audio !")

else:
  with autocast():
    transcript = model.transcribe([audio_path], batch_size=1)[0]

## Write transcription

In [ ]:
with open(transcribed_filepath, 'w', encoding='utf-8') as f:
    f.write(f"{transcript}\n")

# Compute accuracy of transcription

Now that we have a model's transcriped result, we compare the WER and CER against the "ground truth" transcription that we preprocessed earlier

In [ ]:
from nemo.collections.asr.metrics.wer import word_error_rate

In [ ]:
ground_truth = normalized_transcript_path
model_transcript = transcribed_filepath

In [ ]:
with open(ground_truth, 'r') as f:
    ground_truth_txt = f.readlines()
    ground_truth_txt = [text.replace("\n", "") for text in ground_truth_txt]

with open(model_transcript, 'r') as f:
    transcription_txt = f.readlines()
    transcription_txt = [text.replace("\n", "") for text in transcription_txt]

Compute both the character error rate and the word error rate (this might take a while on such large text transcripts)

In [ ]:
cer = word_error_rate(transcription_txt, ground_truth_txt, use_cer=True)
wer = word_error_rate(transcription_txt, ground_truth_txt, use_cer=False)
print("Finished computing metrics !")

In [ ]:
print(f"CER : {cer * 100:0.4f}%")
print(f"WER : {wer * 100:0.4f}%")

-----
The model did fairly well, considering it wasn't trained on any corpus with technical terms (the train corpus is only publically available speech datasets). Furthermore, the ground truth preprocessing is not sufficient in some cases, but for a demonstration it's a reasonable effort.

# Diffing the generated transcript

While a numeric score describes some context of how well the model did, it is much more useful to actually visualize the differences between the ground truth and the model transcription. 

We will partially port https://skeptric.com/python-diffs/ to suit our purposes. Note that we don't have any new lines so minor modifications will be necessary.

In [ ]:
import difflib
from typing import List, Any, Callable, Tuple, Union
from itertools import zip_longest
import html

Token = str
TokenList = List[Token]

In [ ]:
whitespace = re.compile('\s+')
# end_sentence = re.compile('[.!?]\s+')
end_sentence = re.compile('[.]\s+')

def tokenize(s:str) -> TokenList:
    '''Split a string into tokens'''
    return whitespace.split(s)

def untokenize(ts:TokenList) -> str:
    '''Join a list of tokens into a string'''
    return ' '.join(ts)

def sentencize(s:str) -> TokenList:
    '''Split a string into a list of sentences'''
    return end_sentence.split(s)

def unsentencise(ts:TokenList) -> str:
    '''Join a list of sentences into a string'''
    return '. '.join(ts)

def html_unsentencise(ts:TokenList) -> str:
    '''Joing a list of sentences into HTML for display'''
    return ''.join(f'<p>{t}</p>' for t in ts)

In [ ]:
def mark_text(text:str) -> str:
    return f'<span style="color: red;">{text}</span>'
    
def mark_span(text:TokenList) -> TokenList:
    if len(text) > 0:
        text[0] = '<span style="background: #69E2FB;">' + text[0]
        text[-1] += '</span>'
    return text

# def mark_span(text:TokenList) -> TokenList:
#     return [mark_text(token) for token in text]

def markup_diff(a:TokenList, b:TokenList,
                mark=mark_span,
                default_mark = lambda x: x,
                isjunk=None) -> Tuple[TokenList, TokenList]:
    """Returns a and b with any differences processed by mark

    Junk is ignored by the differ
    """
    seqmatcher = difflib.SequenceMatcher(isjunk=isjunk, a=a, b=b, autojunk=False)
    out_a, out_b = [], []
    for tag, a0, a1, b0, b1 in seqmatcher.get_opcodes():
        markup = default_mark if tag == 'equal' else mark
        out_a += markup(a[a0:a1])
        out_b += markup(b[b0:b1])
    assert len(out_a) == len(a)
    assert len(out_b) == len(b)
    return out_a, out_b


def align_seqs(a: TokenList, b: TokenList, fill:Token='') -> Tuple[TokenList, TokenList]:
    out_a, out_b = [], []
    seqmatcher = difflib.SequenceMatcher(a=a, b=b, autojunk=False)
    for tag, a0, a1, b0, b1 in seqmatcher.get_opcodes():
        delta = (a1 - a0) - (b1 - b0)
        out_a += a[a0:a1] + [fill] * max(-delta, 0)
        out_b += b[b0:b1] + [fill] * max(delta, 0)
    assert len(out_a) == len(out_b)
    return out_a, out_b


def html_sidebyside(a, b):
    # Set the panel display
    out = '<div style="display: grid;grid-template-columns: 1fr 1fr;grid-gap: 20px;">'
    # There's some CSS in Jupyter notebooks that makes the first pair unalign. This is a workaround
    out += '<p></p><p></p>'
    for left, right in zip_longest(a, b, fillvalue=''):
        out += f'<p>{left}</p>'
        out += f'<p>{right}</p>'
        out += '</div>'
    return out

def html_diffs(a, b):
    a = html.escape(a)
    b = html.escape(b)

    out_a, out_b = [], []
    for sent_a, sent_b in zip(*align_seqs(sentencize(a), sentencize(b))):
        mark_a, mark_b = markup_diff(tokenize(sent_a), tokenize(sent_b))
        out_a.append(untokenize(mark_a))
        out_b.append(untokenize(mark_b))

    return html_sidebyside(out_a, out_b)


In [ ]:
from IPython.display import HTML, display
def show_diffs(a, b):
    display(HTML(html_diffs(a,b)))

### Side by side comparison

In [ ]:
show_diffs(ground_truth_txt[0], transcription_txt[0])

### Inplace comparison

In [ ]:
!pip install diff_match_patch

In [ ]:
import diff_match_patch

def show_inplace_diff(ground_truth, generated_transcript):
  print("Showing inplace diff")
  print("Ground truth is in red, predicted word is in green")
  
  print()
  print("-" * 160)
  print()

  diff = diff_match_patch.diff_match_patch()
  diff.Diff_Timeout = 0
  orig_enc, pred_enc, enc = diff.diff_linesToChars(ground_truth.replace(" ", "\n"), generated_transcript.replace(" ", "\n"))
  diffs = diff.diff_main(orig_enc, pred_enc, False)
  diff.diff_charsToLines(diffs, enc)
  diffs_post = []
  for d in diffs:
      diffs_post.append((d[0], d[1].replace('\n', ' ')))

  diff_html = diff.diff_prettyHtml(diffs_post)
  return HTML(diff_html)

In [ ]:
show_inplace_diff(ground_truth_txt[0], transcription_txt[0])

# [Extra] Seeking the upper limit of audio sequence length

So we were able to transcribe a nearly 40 minute podcast and obtain a moderately accurate transcript. While this was great for a first effort, this raises the question - 

**Given 32 GB of memory on a GPU, what is the upper bound of audio duration that can be transcribed by a Citrinet model in a single forward pass?**

In [ ]:
import librosa
import datetime
import math
import gc

original_duration = librosa.get_duration(filename=audio_path)
print("Original audio duration :", datetime.timedelta(seconds=original_duration))

In order to extend the audio duration, we will concatenate the same audio clip multiple times, and then trim off any excess duration from the clip as needed.

For convenience, we provide a scalar multiplier to the original audio duration.

In [ ]:
# concatenate the file multiple times
NUM_REPEATS = 3.5
new_duration = original_duration * NUM_REPEATS

# write a temp file
with open('audio_repeat.txt', 'w') as f:
  for _ in range(int(math.ceil(NUM_REPEATS))):
    f.write(f"file {audio_path}\n")

Duplicate the audio several times, then trim off the required duration from the concatenated audio clip.

In [ ]:
repeated_audio_path = "data/processed/concatenated_audio.wav"

!ffmpeg -t {new_duration} -f concat -i audio_repeat.txt -c copy -t {new_duration} {repeated_audio_path} -y
print("Finished repeating audio file!")

In [ ]:
original_duration = librosa.get_duration(filename=audio_path)
repeated_duration = librosa.get_duration(filename=repeated_audio_path)

print("Original audio duration :", datetime.timedelta(seconds=original_duration))
print("Repeated audio duration :", datetime.timedelta(seconds=repeated_duration))

Attempt to transcribe it (Note this may OOM!)

In [ ]:
# Clear up memory
torch.cuda.empty_cache()
gc.collect()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'  # You can transcribe even longer samples on the CPU, though it will take much longer !
model = model.to(device)

In [ ]:
%%time

with autocast():
    transcript_repeated = model.transcribe([repeated_audio_path], batch_size=1)[0]
    del transcript_repeated

Given a large amount of GPU memory, the Citrinet model can efficiently transcribe long audio segments with ease, without the need for streaming inference.

This is possible due to a simple reason - no attention mechanism is used, and Squeeze-and-Excitation mechanism does not require quadratic memory requirements yet still provides reasonable global context information.